In [6]:
import pandas as pd
import random
import time
from apiclient.errors import HttpError
import pymysql
import mysql.connector
from sqlalchemy import create_engine
from config import uname, pword, location, host

engine = create_engine(f'mysql+pymysql://{uname}:{pword}@{location}/analytics', echo=False)

In [7]:
seodf = pd.read_csv("SEO_Database.csv")
seodf.head()

,18 digit Property ID,ARC Program: ID,ARC Program: Program ID,Billing Code (AccPac),Operations Region,Property Name (formula),SEO Cost,Program Official Start,UA Code,View ID,Current Property Owner,Units
0,a072E00000XZc4rQAD,a2i2E000004uiwl,PGM-87640,2086.0,Central,Parkside Bella Terra,300,4/25/2019,UA-50039503-21,194469930,LivCor,342
1,a072E00000XZYr2QAH,a2i2E000004uiYt,PGM-87375,2077.0,Central,801 LasCo.,600,4/26/2019,UA-83547238-13,194325110,Pacific Life Insurance Company,422
2,a072E00000XZa9VQAT,a2i2E000004uiY2,PGM-87322,2083.0,Central,Archstone Toscano,900,4/25/2019,UA-50039503-18,194371468,Alliance Residential Company,474
3,a078000000VDZpwAAH,a2i80000003UAzX,PGM-69896,1863.0,Central,Broadstone Traditions,600,9/1/2018,UA-50039503-7,175502689,Alliance Residential Company,261
4,a078000000QDWHNAA5,a2i80000003Uysn,PGM-77607,1494.0,Central,Broadstone Woodmill Creek,300,6/29/2018,UA-69155700-1,178632607,UBS Realty Investors LLC,380


In [8]:
seodf_str = seodf.astype({"View ID":str})
view_id_list = seodf_str["View ID"].tolist()
view_sample = view_id_list[0:5]
# view_sample
len(view_id_list)

578

# Loop Through List of Accounts
> ## Date Range is selected here

> ## Set Date {"Month Year"}. 
Spell out month <br>
Example:
> > date = "January 2018"
    
    Note: This only applies to a full calendar month of reporting, all other date ranges must be hard coded

In [9]:
date = "April 2018"

In [10]:
# Time Dependencies
import arrow
import datetime

# This cell converts the above date into a format that can be passed into the Analytics API only requiring date to be set once. Additionally it names the Excel output file

# Capture first and last date of month/year selected
arrow_start_date = arrow.get(date,'MMMM YYYY').floor('month')
arrow_end_date = arrow.get(date,'MMMM YYYY').ceil('month')

# convert arrow object to datetime object
start_date_obj = arrow_start_date.date()
end_date_obj = arrow_end_date.date()

# convert datetime object to string
start_date = start_date_obj.strftime('%Y-%m-%d')
end_date = end_date_obj.strftime('%Y-%m-%d')

print(start_date)
print(end_date)

2018-04-01
2018-04-30


In [11]:
"""Hello Analytics Reporting API V4."""

import argparse

from apiclient.discovery import build
import httplib2
from oauth2client import client
from oauth2client import file
from oauth2client import tools

SCOPES = ['https://www.googleapis.com/auth/analytics.readonly']
DISCOVERY_URI = ('https://analyticsreporting.googleapis.com/$discovery/rest')
CLIENT_SECRETS_PATH = 'client_secrets.json' # Path to client_secrets.json file.
sessions=[]

def initialize_analyticsreporting():
  """Initializes the analyticsreporting service object.

  Returns:
    analytics an authorized analyticsreporting service object.
  """
  # Parse command-line arguments.
  parser = argparse.ArgumentParser(
      formatter_class=argparse.RawDescriptionHelpFormatter,
      parents=[tools.argparser])
  flags = parser.parse_args([])

  # Set up a Flow object to be used if we need to authenticate.
  flow = client.flow_from_clientsecrets(
      CLIENT_SECRETS_PATH, scope=SCOPES,
      message=tools.message_if_missing(CLIENT_SECRETS_PATH))

  # Prepare credentials, and authorize HTTP object with them.
  # If the credentials don't exist or are invalid run through the native client
  # flow. The Storage object will ensure that if successful the good
  # credentials will get written back to a file.
  storage = file.Storage('analyticsreporting.dat')
  credentials = storage.get()
  if credentials is None or credentials.invalid:
    credentials = tools.run_flow(flow, storage, flags)
  http = credentials.authorize(http=httplib2.Http())

  # Build the service object.
  analytics = build('analytics', 'v4', http=http, discoveryServiceUrl=DISCOVERY_URI)

  return analytics

df_list = []

error_date = "No Data Exists"
error_date2 = "Campaign Inactive"
    
def get_report(analytics):
    print(f'View ID is {VIEW_ID}')
  # Use the Analytics Service Object to query the Analytics Reporting API V4.
    return analytics.reports().batchGet(
      body={
        'reportRequests': [
        {
          'viewId': VIEW_ID,
          'dateRanges': [{'startDate': start_date, 'endDate': end_date}],
          'metrics': [{'expression': 'ga:sessions'},{'expression': 'ga:goalCompletionsAll'}, {'expression': 'ga:sessionDuration'}, {'expression': 'ga:bounces'}, {'expression': 'ga:goalCompletionsAll'}]
#           'dimensions':[{'name': 'ga:channelGrouping'}]   
        }]
      }
    ).execute()

def print_response(response):
    """Parses and prints the Analytics Reporting API V4 response"""

    try:
        for report in response.get('reports', []):
            columnHeader = report.get('columnHeader', {})
            dimensionHeaders = columnHeader.get('dimensions', [])
            metricHeaders = columnHeader.get('metricHeader', {}).get('metricHeaderEntries', [])
            rows = report.get('data', {}).get('rows', [])
            #     create empty list to later append dictionaries into. 

            for row in rows:
                dimensions = row.get('dimensions', [])
                dateRangeValues = row.get('metrics', [])
        #         Create empty dictionary which will contain key,value pairs for relevant metrics
                mt_dict = dict({})
                mt_dict["View ID"] = VIEW_ID
                mt_dict["Start Date"] = start_date
                mt_dict["End Date"] = end_date
                for header, dimension in zip(dimensionHeaders, dimensions):
    #                 print (header + ': ' + dimension)
        #             Create an entry in the dictionary with the Dimension as the Key (splitting for formatting) and the dimension value as the value
                    mt_dict[header.split(":")[-1]] = dimension
                for i, values in enumerate(dateRangeValues):
    #                 print ('Date range (' + str(i) + ')')
                    for metricHeader, value in zip(metricHeaders, values.get('values')):
    #                     print (metricHeader.get('name') + ': ' + value)
        #                 Create an entry in the dictionary with the Metric as the key and the metric value as the value
                        mt_dict[metricHeader.get('name').split(":")[-1]] = value

        # Append dictionary for each iteration into list
                df_list.append(mt_dict)

        print(f'{VIEW_ID} complete')
        # Return a DataFrame object that is created from the above list
        return pd.DataFrame(df_list)
    except AttributeError:
        mt_dict = dict({})
        mt_dict["View ID"] = VIEW_ID
        mt_dict["Start Date"] = error_date
        mt_dict["End Date"] = error_date2
        df_list.append(mt_dict)


def makeRequestWithExponentialBackoff(analytics):

    for n in range(1, 6):
        try:
            return get_report(analytics)            

        except HttpError as error:
            if error.resp.reason in ['userRateLimitExceeded', 'quotaExceeded',
                               'internalServerError', 'backendError','Too Many Requests']:
                time.sleep((2.52 ** n) + random.random())
            else:
                print(error.resp.reason)
                break

            print(f"Request {n} FAILURE")
            print(f"Wait {2.52 ** n} second")

def main():

    analytics = initialize_analyticsreporting()
    response = makeRequestWithExponentialBackoff(analytics)
    df = print_response(response)
    return df

for VIEW_ID in view_id_list:
    main()


# Include below if using this in a .py File
# if __name__ == '__main__':
#   main()

View ID is 194469930
194469930 complete
View ID is 194325110
194325110 complete
View ID is 194371468
194371468 complete
View ID is 175502689
175502689 complete
View ID is 178632607
178632607 complete
View ID is 171726956
171726956 complete
View ID is 181634063
181634063 complete
View ID is 188832488
188832488 complete
View ID is 177071738
177071738 complete
View ID is 176395500
176395500 complete
View ID is 178744348
178744348 complete
View ID is 175521826
175521826 complete
View ID is 174632823
174632823 complete
View ID is 177352497
177352497 complete
View ID is 178491222
178491222 complete
View ID is 177301433
177301433 complete
View ID is 193884885
193884885 complete
View ID is 174148543
174148543 complete
View ID is 184768504
184768504 complete
View ID is 178509418
178509418 complete
View ID is 178504572
178504572 complete
View ID is 176672309
176672309 complete
View ID is 169760743
169760743 complete
View ID is 178518108
178518108 complete
View ID is 178719280
178719280 complete


185004655 complete
View ID is 183123320
183123320 complete
View ID is 172997073
172997073 complete
View ID is 187442847
187442847 complete
View ID is 173671593
173671593 complete
View ID is 183807538
183807538 complete
View ID is 184053793
184053793 complete
View ID is 186292070
186292070 complete
View ID is 182339851
182339851 complete
View ID is 173655048
173655048 complete
View ID is 173051308
173051308 complete
View ID is 173646433
173646433 complete
View ID is 182748778
182748778 complete
View ID is 173623667
173623667 complete
View ID is 186861426
186861426 complete
View ID is 174900137
Request 1 FAILURE
Wait 2.52 second
View ID is 174900137
174900137 complete
View ID is 191620297
191620297 complete
View ID is 190400492
190400492 complete
View ID is 171910648
171910648 complete
View ID is 183663655
183663655 complete
View ID is 184063003
184063003 complete
View ID is 183662617
183662617 complete
View ID is 184076777
184076777 complete
View ID is 183971359
183971359 complete
View 

Request 1 FAILURE
Wait 2.52 second
View ID is 142896479
Request 2 FAILURE
Wait 6.3504000000000005 second
View ID is 142896479
Request 3 FAILURE
Wait 16.003008 second
View ID is 142896479
142896479 complete
View ID is 178510127
178510127 complete
View ID is 179689764
179689764 complete
View ID is 188069832
188069832 complete
View ID is 94297001
Request 1 FAILURE
Wait 2.52 second
View ID is 94297001
94297001 complete
View ID is 172087276
Request 1 FAILURE
Wait 2.52 second
View ID is 172087276
172087276 complete
View ID is 173696064
173696064 complete
View ID is 172070620
Request 1 FAILURE
Wait 2.52 second
View ID is 172070620
Request 2 FAILURE
Wait 6.3504000000000005 second
View ID is 172070620
172070620 complete
View ID is 92077736
Request 1 FAILURE
Wait 2.52 second
View ID is 92077736
92077736 complete
View ID is 153639282
Request 1 FAILURE
Wait 2.52 second
View ID is 153639282
153639282 complete
View ID is 153642414
153642414 complete
View ID is 146245070
146245070 complete
View ID is

180246757 complete
View ID is 178098620
178098620 complete
View ID is 171182952
Request 1 FAILURE
Wait 2.52 second
View ID is 171182952
Request 2 FAILURE
Wait 6.3504000000000005 second
View ID is 171182952
171182952 complete
View ID is 188086309
188086309 complete
View ID is 158779062
158779062 complete
View ID is 158753732
158753732 complete
View ID is 191835145
191835145 complete
View ID is 192419096
192419096 complete


In [12]:
# Identidies how many successful calls there were
len(df_list)

218

# Writing Output to DataFrame

In [13]:
result_df = pd.DataFrame.from_dict(df_list)
result_df["View ID"] = pd.to_numeric(result_df["View ID"])
result_df["bounces"] = pd.to_numeric(result_df["bounces"])
result_df["goalCompletionsAll"] = pd.to_numeric(result_df["goalCompletionsAll"])
result_df["sessionDuration"] = pd.to_numeric(result_df["sessionDuration"])
result_df["sessions"] = pd.to_numeric(result_df["sessions"])
# result_df.head()
result_df.dtypes

End Date               object
Start Date             object
View ID                 int64
bounces               float64
goalCompletionsAll    float64
sessionDuration       float64
sessions              float64
dtype: object

# Merge DataFrames


In [14]:
# # WITH CHANNEL GROUPINGS
# merged = seodf.merge(result_df,how="inner", on="View ID")
# merged = merged[['Start Date', 'End Date','18 digit Property ID', 'ARC Program: ID', 'ARC Program: Program ID',
#        'Billing Code (AccPac)', 'Operations Region', 'Property Name (formula)',
#        'SEO Cost', 'Program Official Start', 'UA Code', 
#        'Current Property Owner', 'Units', 'View ID', 'channelGrouping' , 'bounces',
#        'goalCompletionsAll', 'sessionDuration', 'sessions']]

# merged = merged.rename(columns={"bounces": "Bounces",
#                                "goalCompletionsAll": "All Goal Completions",
#                                "sessionDuration": "Session Duration(Seconds)",
#                                "channelGrouping": "Channel Group"})

# merged["Average Session Duration"] = merged["Session Duration(Seconds)"]/merged["sessions"]
# merged["Conversion Rate"] = merged["All Goal Completions"]/merged["sessions"]
# merged["Bounce Rate"] = merged["Bounces"]/merged["sessions"]
# merged.head()



# WITHOUT CHANNEL GROUPINGS
merged = seodf.merge(result_df,how="inner", on="View ID")
merged = merged[['Start Date', 'End Date','18 digit Property ID', 'ARC Program: ID', 'ARC Program: Program ID',
       'Billing Code (AccPac)', 'Operations Region', 'Property Name (formula)',
       'SEO Cost', 'Program Official Start', 'UA Code', 
       'Current Property Owner', 'Units', 'View ID', 'bounces',
       'goalCompletionsAll', 'sessionDuration', 'sessions']]

merged = merged.rename(columns={"bounces": "Bounces",
                               "goalCompletionsAll": "All Goal Completions",
                               "sessionDuration": "Session Duration(Seconds)",
                               "channelGrouping": "Channel Group"})

merged["Average Session Duration"] = merged["Session Duration(Seconds)"]/merged["sessions"]
merged["Conversion Rate"] = merged["All Goal Completions"]/merged["sessions"]
merged["Bounce Rate"] = merged["Bounces"]/merged["sessions"]
merged.head()

,Start Date,End Date,18 digit Property ID,ARC Program: ID,ARC Program: Program ID,Billing Code (AccPac),Operations Region,Property Name (formula),SEO Cost,Program Official Start,...,Current Property Owner,Units,View ID,Bounces,All Goal Completions,Session Duration(Seconds),sessions,Average Session Duration,Conversion Rate,Bounce Rate
0,2018-04-01,2018-04-30,a078000000VFYsIAAX,a2i80000003U9Pz,PGM-69083,1818.0,Central,Broadstone Briar Forest,1200,3/16/2018,...,Alliance Residential Company,342,171726956,972.0,198.0,368806.0,2243.0,164.425323,0.088275,0.433348
1,2018-04-01,2018-04-30,a078000000P1bxfAAB,a2i800000048Gzg,PGM-37767,1211.0,Central,The Oaks at Techridge,300,9/5/2014,...,"Belkorp Industries, Inc.",336,174148543,141.0,0.0,53378.0,439.0,121.589977,0.000000,0.321185
2,2018-04-01,2018-04-30,a078000000SpiwJAAR,a2i800000048Gws,PGM-37593,1595.0,Central,Benton Pointe,600,2/13/2018,...,Rivendell Global Real Estate Inc.,456,169760743,1149.0,445.0,592766.0,3289.0,180.226817,0.135299,0.349346
3,2018-04-01,2018-04-30,a0780000003RyqDAAS,a2i800000048H2d,PGM-37950,427.0,Central,Broadstone New Territory II,300,12/10/2015,...,Harvest Equities,248,149547335,751.0,264.0,409893.0,1952.0,209.986168,0.135246,0.384734
4,2018-04-01,2018-04-30,a078000000PhYk2AAF,a2i800000048Gz8,PGM-37733,1291.0,Central,Asher,300,4/15/2015,...,Castle Lanterra Properties,452,174154707,434.0,3.0,189435.0,1260.0,150.345238,0.002381,0.344444


In [15]:
merged = merged.rename(columns={"sessions": "Website Sessions",
                               "Average Session Duration": "Avg_Time_On_Site"})

merged["Bounce Rate"] = pd.Series(["{0:.2f}%".format(val * 100) for val in merged["Bounce Rate"]], index = merged.index)
merged["Conversion Rate"] = pd.Series(["{0:.2f}%".format(val * 100) for val in merged["Conversion Rate"]], index = merged.index)
merged['Avg_Time_On_Site'] = pd.to_datetime(merged["Avg_Time_On_Site"], unit='m')
merged['Avg_Time_On_Site'] = merged['Avg_Time_On_Site'].astype(str)
merged['Avg_Time_On_Site'] = (merged.Avg_Time_On_Site.str.split(' ').str[1])
merged['Avg_Time_On_Site'] = (merged.Avg_Time_On_Site.str.split('.').str[0])
merged['Avg_Time_On_Site'] = (merged.Avg_Time_On_Site.str.rsplit(':',1).str[0])

merged = merged.rename(columns={"Avg_Time_On_Site": "Average Session Duration"})

In [16]:
merged.head()

,Start Date,End Date,18 digit Property ID,ARC Program: ID,ARC Program: Program ID,Billing Code (AccPac),Operations Region,Property Name (formula),SEO Cost,Program Official Start,...,Current Property Owner,Units,View ID,Bounces,All Goal Completions,Session Duration(Seconds),Website Sessions,Average Session Duration,Conversion Rate,Bounce Rate
0,2018-04-01,2018-04-30,a078000000VFYsIAAX,a2i80000003U9Pz,PGM-69083,1818.0,Central,Broadstone Briar Forest,1200,3/16/2018,...,Alliance Residential Company,342,171726956,972.0,198.0,368806.0,2243.0,02:44,8.83%,43.33%
1,2018-04-01,2018-04-30,a078000000P1bxfAAB,a2i800000048Gzg,PGM-37767,1211.0,Central,The Oaks at Techridge,300,9/5/2014,...,"Belkorp Industries, Inc.",336,174148543,141.0,0.0,53378.0,439.0,02:01,0.00%,32.12%
2,2018-04-01,2018-04-30,a078000000SpiwJAAR,a2i800000048Gws,PGM-37593,1595.0,Central,Benton Pointe,600,2/13/2018,...,Rivendell Global Real Estate Inc.,456,169760743,1149.0,445.0,592766.0,3289.0,03:00,13.53%,34.93%
3,2018-04-01,2018-04-30,a0780000003RyqDAAS,a2i800000048H2d,PGM-37950,427.0,Central,Broadstone New Territory II,300,12/10/2015,...,Harvest Equities,248,149547335,751.0,264.0,409893.0,1952.0,03:29,13.52%,38.47%
4,2018-04-01,2018-04-30,a078000000PhYk2AAF,a2i800000048Gz8,PGM-37733,1291.0,Central,Asher,300,4/15/2015,...,Castle Lanterra Properties,452,174154707,434.0,3.0,189435.0,1260.0,02:30,0.24%,34.44%


In [17]:
merged = merged[["Start Date", "End Date","Website Sessions","Session Duration(Seconds)","Average Session Duration","All Goal Completions","Conversion Rate","Bounces","Bounce Rate",
                 "Property Name (formula)","Current Property Owner","Operations Region","18 digit Property ID", "Billing Code (AccPac)"]]

merged["Website Sessions"] = pd.Series(["{0:.0f}".format(val * 1) for val in merged["Website Sessions"]], index = merged.index)
merged["Billing Code (AccPac)"] = pd.Series(["{0:.0f}".format(val * 1) for val in merged["Billing Code (AccPac)"]], index = merged.index)
merged.head()

,Start Date,End Date,Website Sessions,Session Duration(Seconds),Average Session Duration,All Goal Completions,Conversion Rate,Bounces,Bounce Rate,Property Name (formula),Current Property Owner,Operations Region,18 digit Property ID,Billing Code (AccPac)
0,2018-04-01,2018-04-30,2243,368806.0,02:44,198.0,8.83%,972.0,43.33%,Broadstone Briar Forest,Alliance Residential Company,Central,a078000000VFYsIAAX,1818
1,2018-04-01,2018-04-30,439,53378.0,02:01,0.0,0.00%,141.0,32.12%,The Oaks at Techridge,"Belkorp Industries, Inc.",Central,a078000000P1bxfAAB,1211
2,2018-04-01,2018-04-30,3289,592766.0,03:00,445.0,13.53%,1149.0,34.93%,Benton Pointe,Rivendell Global Real Estate Inc.,Central,a078000000SpiwJAAR,1595
3,2018-04-01,2018-04-30,1952,409893.0,03:29,264.0,13.52%,751.0,38.47%,Broadstone New Territory II,Harvest Equities,Central,a0780000003RyqDAAS,427
4,2018-04-01,2018-04-30,1260,189435.0,02:30,3.0,0.24%,434.0,34.44%,Asher,Castle Lanterra Properties,Central,a078000000PhYk2AAF,1291


In [18]:
merged.count()

Start Date                   256
End Date                     256
Website Sessions             256
Session Duration(Seconds)    255
Average Session Duration     255
All Goal Completions         255
Conversion Rate              256
Bounces                      255
Bounce Rate                  256
Property Name (formula)      256
Current Property Owner       256
Operations Region            256
18 digit Property ID         256
Billing Code (AccPac)        256
dtype: int64

In [19]:
# REMOVE ALL ROWS WITH NO DATA
active = merged[merged['Start Date']!="No Data Exists"]
active.drop_duplicates(inplace=True)
active.reset_index(drop=True, inplace=True)
active.head()

C:\Users\habet\Anaconda3\envs\PythonData\lib\site-packages\ipykernel_launcher.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  This is separate from the ipykernel package so we can avoid doing imports until


,Start Date,End Date,Website Sessions,Session Duration(Seconds),Average Session Duration,All Goal Completions,Conversion Rate,Bounces,Bounce Rate,Property Name (formula),Current Property Owner,Operations Region,18 digit Property ID,Billing Code (AccPac)
0,2018-04-01,2018-04-30,2243,368806.0,02:44,198.0,8.83%,972.0,43.33%,Broadstone Briar Forest,Alliance Residential Company,Central,a078000000VFYsIAAX,1818
1,2018-04-01,2018-04-30,439,53378.0,02:01,0.0,0.00%,141.0,32.12%,The Oaks at Techridge,"Belkorp Industries, Inc.",Central,a078000000P1bxfAAB,1211
2,2018-04-01,2018-04-30,3289,592766.0,03:00,445.0,13.53%,1149.0,34.93%,Benton Pointe,Rivendell Global Real Estate Inc.,Central,a078000000SpiwJAAR,1595
3,2018-04-01,2018-04-30,1952,409893.0,03:29,264.0,13.52%,751.0,38.47%,Broadstone New Territory II,Harvest Equities,Central,a0780000003RyqDAAS,427
4,2018-04-01,2018-04-30,1260,189435.0,02:30,3.0,0.24%,434.0,34.44%,Asher,Castle Lanterra Properties,Central,a078000000PhYk2AAF,1291


In [20]:
active.count()

Start Date                   206
End Date                     206
Website Sessions             206
Session Duration(Seconds)    206
Average Session Duration     206
All Goal Completions         206
Conversion Rate              206
Bounces                      206
Bounce Rate                  206
Property Name (formula)      206
Current Property Owner       206
Operations Region            206
18 digit Property ID         206
Billing Code (AccPac)        206
dtype: int64

In [21]:
month = date.split(" ")[0]
year = date.split(" ")[1]
active.to_csv(f"Reports/{month} {year} Data.csv")

# Write to MySQL Database

In [22]:
# Push DataFrame to SQL Database
active.to_sql(name='data',con=engine,if_exists='append', index=False)

# Read in Full MySQL Database

In [23]:
db_connection = mysql.connector.connect(host='localhost',database='analytics',user=uname ,password=pword)
full_df = pd.read_sql('SELECT * FROM data', con=db_connection)

In [24]:
full_df

,Start Date,End Date,Website Sessions,Session Duration(Seconds),Average Session Duration,All Goal Completions,Conversion Rate,Bounces,Bounce Rate,Property Name (formula),Current Property Owner,Operations Region,18 digit Property ID,Billing Code (AccPac)
0,2019-04-01,2019-04-30,691,185046.0,04:27,25,3.62%,249,36.03%,801 LasCo.,Pacific Life Insurance Company,Central,a072E00000XZYr2QAH,2077
1,2019-04-01,2019-04-30,128,30604.0,03:59,47,36.72%,34,26.56%,Archstone Toscano,Alliance Residential Company,Central,a072E00000XZa9VQAT,2083
2,2019-04-01,2019-04-30,4635,727424.0,02:36,190,4.10%,2005,43.26%,Broadstone Traditions,Alliance Residential Company,Central,a078000000VDZpwAAH,1863
3,2019-04-01,2019-04-30,4202,522836.0,02:04,254,6.04%,1952,46.45%,Broadstone Woodmill Creek,UBS Realty Investors LLC,Central,a078000000QDWHNAA5,1494
4,2019-04-01,2019-04-30,2661,433831.0,02:43,270,10.15%,1089,40.92%,Broadstone Briar Forest,Alliance Residential Company,Central,a078000000VFYsIAAX,1818
5,2019-04-01,2019-04-30,5877,980701.0,02:46,160,2.72%,1618,27.53%,Broadstone Lofts at Hermann Park,Alliance Residential Company,Central,a078000000W63hSAAR,1871
6,2019-04-01,2019-04-30,1986,116489.0,00:58,10,0.50%,1784,89.83%,Broadstone Barker Cypress,Alliance Residential Company,Central,a078000000W7UhhAAF,2043
7,2019-04-01,2019-04-30,4563,633630.0,02:18,149,3.27%,1533,33.60%,La Costa Villas,Boston Capital,Central,a078000000W5dgOAAR,1857
8,2019-04-01,2019-04-30,3460,444259.0,02:08,516,14.91%,1070,30.92%,The Villages of Briar Forest,White Oak Partners,Central,a078000000W5nyoAAB,2115
9,2019-04-01,2019-04-30,5946,778306.0,02:10,366,6.16%,2643,44.45%,Stella at Riverstone,Cardone Capital,Central,a078000000Tv45SAAR,1988


# Central Region

In [470]:
central = full_df[full_df['Operations Region']=="Central"]
central

,Start Date,End Date,Website Sessions,Session Duration(Seconds),Average Session Duration,All Goal Completions,Conversion Rate,Bounces,Bounce Rate,Property Name (formula),Current Property Owner,Operations Region,18 digit Property ID,Billing Code (AccPac)
0,2019-04-01,2019-04-30,691,185046.0,04:27,25,3.62%,249,36.03%,801 LasCo.,Pacific Life Insurance Company,Central,a072E00000XZYr2QAH,2077
1,2019-04-01,2019-04-30,128,30604.0,03:59,47,36.72%,34,26.56%,Archstone Toscano,Alliance Residential Company,Central,a072E00000XZa9VQAT,2083
2,2019-04-01,2019-04-30,4635,727424.0,02:36,190,4.10%,2005,43.26%,Broadstone Traditions,Alliance Residential Company,Central,a078000000VDZpwAAH,1863
3,2019-04-01,2019-04-30,4202,522836.0,02:04,254,6.04%,1952,46.45%,Broadstone Woodmill Creek,UBS Realty Investors LLC,Central,a078000000QDWHNAA5,1494
4,2019-04-01,2019-04-30,2661,433831.0,02:43,270,10.15%,1089,40.92%,Broadstone Briar Forest,Alliance Residential Company,Central,a078000000VFYsIAAX,1818
5,2019-04-01,2019-04-30,5877,980701.0,02:46,160,2.72%,1618,27.53%,Broadstone Lofts at Hermann Park,Alliance Residential Company,Central,a078000000W63hSAAR,1871
6,2019-04-01,2019-04-30,1986,116489.0,00:58,10,0.50%,1784,89.83%,Broadstone Barker Cypress,Alliance Residential Company,Central,a078000000W7UhhAAF,2043
7,2019-04-01,2019-04-30,4563,633630.0,02:18,149,3.27%,1533,33.60%,La Costa Villas,Boston Capital,Central,a078000000W5dgOAAR,1857
8,2019-04-01,2019-04-30,3460,444259.0,02:08,516,14.91%,1070,30.92%,The Villages of Briar Forest,White Oak Partners,Central,a078000000W5nyoAAB,2115
9,2019-04-01,2019-04-30,5946,778306.0,02:10,366,6.16%,2643,44.45%,Stella at Riverstone,Cardone Capital,Central,a078000000Tv45SAAR,1988


In [513]:
central_clean = central[["Start Date","Website Sessions", "Session Duration(Seconds)","Bounces","All Goal Completions"]]
central_clean["Website Sessions"] = central_clean["Website Sessions"].astype(int)
central_group = central_clean.groupby('Start Date', as_index=True)
central_sum = central_group.sum()
central_sum.head()


C:\Users\habet\Anaconda3\envs\PythonData\lib\site-packages\ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  


,Website Sessions,Session Duration(Seconds),Bounces,All Goal Completions
Start Date,,,,
2018-05-01,73194,11038338.0,27654,4502
2018-06-01,112531,18234069.0,41345,8745
2018-07-01,200989,31400566.0,73762,18506
2018-08-01,256579,37896274.0,100286,22255
2018-09-01,225555,31234386.0,87830,19474


In [511]:
# central_sum["Goal Completion Rate"] = central_sum[""]

In [477]:
full_df.to_csv(f"Reports/Full Year Report.csv")

In [476]:
full_df

,Start Date,End Date,Website Sessions,Session Duration(Seconds),Average Session Duration,All Goal Completions,Conversion Rate,Bounces,Bounce Rate,Property Name (formula),Current Property Owner,Operations Region,18 digit Property ID,Billing Code (AccPac)
0,2019-04-01,2019-04-30,691,185046.0,04:27,25,3.62%,249,36.03%,801 LasCo.,Pacific Life Insurance Company,Central,a072E00000XZYr2QAH,2077
1,2019-04-01,2019-04-30,128,30604.0,03:59,47,36.72%,34,26.56%,Archstone Toscano,Alliance Residential Company,Central,a072E00000XZa9VQAT,2083
2,2019-04-01,2019-04-30,4635,727424.0,02:36,190,4.10%,2005,43.26%,Broadstone Traditions,Alliance Residential Company,Central,a078000000VDZpwAAH,1863
3,2019-04-01,2019-04-30,4202,522836.0,02:04,254,6.04%,1952,46.45%,Broadstone Woodmill Creek,UBS Realty Investors LLC,Central,a078000000QDWHNAA5,1494
4,2019-04-01,2019-04-30,2661,433831.0,02:43,270,10.15%,1089,40.92%,Broadstone Briar Forest,Alliance Residential Company,Central,a078000000VFYsIAAX,1818
5,2019-04-01,2019-04-30,5877,980701.0,02:46,160,2.72%,1618,27.53%,Broadstone Lofts at Hermann Park,Alliance Residential Company,Central,a078000000W63hSAAR,1871
6,2019-04-01,2019-04-30,1986,116489.0,00:58,10,0.50%,1784,89.83%,Broadstone Barker Cypress,Alliance Residential Company,Central,a078000000W7UhhAAF,2043
7,2019-04-01,2019-04-30,4563,633630.0,02:18,149,3.27%,1533,33.60%,La Costa Villas,Boston Capital,Central,a078000000W5dgOAAR,1857
8,2019-04-01,2019-04-30,3460,444259.0,02:08,516,14.91%,1070,30.92%,The Villages of Briar Forest,White Oak Partners,Central,a078000000W5nyoAAB,2115
9,2019-04-01,2019-04-30,5946,778306.0,02:10,366,6.16%,2643,44.45%,Stella at Riverstone,Cardone Capital,Central,a078000000Tv45SAAR,1988
